In [14]:
import numpy as np
import pandas as pd
import pulp
import matplotlib.pyplot as plt
from scipy.stats import norm
from deap import base, creator, tools, algorithms
import random



### Step 1: Define Energy Sources


In [16]:

sources = ['Solar', 'Wind', 'Gas', 'Hydro', 'Coal', 'Nuclear']
costs = {'Solar': 40, 'Wind': 35, 'Gas': 70, 'Hydro': 50, 'Coal': 60, 'Nuclear': 80}  # $/MWh
capacity = {'Solar': 100, 'Wind': 120, 'Gas': 150, 'Hydro': 80, 'Coal': 130, 'Nuclear': 90}  # Max MW

demand = 300  # Total required demand (MW)


### Step 2: Monte Carlo Simulation for Grid Reliability


In [19]:

def monte_carlo_reliability(trials=10000):
    failures = 0
    for _ in range(trials):
        available_capacity = sum(np.random.normal(capacity[src], 10) for src in sources)
        if available_capacity < demand:
            failures += 1
    return 1 - (failures / trials)

grid_reliability = monte_carlo_reliability()
print(f"Grid Reliability: {grid_reliability:.4f}")


Grid Reliability: 1.0000


### Step 3: Optimization Model (Linear Programming)


In [22]:

prob = pulp.LpProblem("Energy_Optimization", pulp.LpMinimize)
allocation = {src: pulp.LpVariable(src, 0, capacity[src]) for src in sources}

# Objective: Minimize cost
prob += pulp.lpSum(allocation[src] * costs[src] for src in sources)

# Constraint: Meet demand
prob += pulp.lpSum(allocation[src] for src in sources) >= demand

# Solve
prob.solve()

print("\nOptimal Energy Mix:")
for src in sources:
    print(f"{src}: {allocation[src].varValue:.2f} MW")



Optimal Energy Mix:
Solar: 100.00 MW
Wind: 120.00 MW
Gas: 0.00 MW
Hydro: 80.00 MW
Coal: 0.00 MW
Nuclear: 0.00 MW


### Step 4: Genetic Algorithm for Alternative Optimization


In [25]:

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, max(capacity.values()))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=len(sources))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_energy_mix(individual):
    cost = sum(individual[i] * costs[src] for i, src in enumerate(sources))
    penalty = max(0, demand - sum(individual)) * 1000
    return cost + penalty,

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_energy_mix)

pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=None, halloffame=hof, verbose=False)

print("\nOptimized Energy Mix (Genetic Algorithm):")
for i, src in enumerate(sources):
    print(f"{src}: {hof[0][i]:.2f} MW")


Optimized Energy Mix (Genetic Algorithm):
Solar: 66.27 MW
Wind: 175.53 MW
Gas: 6.96 MW
Hydro: 63.78 MW
Coal: 22.07 MW
Nuclear: -32.44 MW
